In [35]:
# 라이브러리 import
! pip install bs4
! pip install pandas
import pandas as pd
from bs4 import BeautifulSoup
import requests
import time
import pprint
import os


[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python3.11 -m pip install --upgrade pip

[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python3.11 -m pip install --upgrade pip


In [36]:

### 중앙부처 데이터만 저장할 수 있도록 만든 코드입니다. 지자체나 지역별 데이터 저장을 원하면 수정하여 활용하시길 바랍니다.
## 1. 온통 청년 API로 정책 html 불러오기
key = '<YOUR_ONTONG_API_KEY>' ### 여기에 신청한 본인의 key를 넣어야 합니다!
page = ['0', '1', '2', '3', '4', '5'] # 조회할 페이지, 총 정책 수가 업데이트로 인해 늘어나면 현재 '0' ~ '4'인 페이지를 추가해야 할 수도 있음. 예) page = ['0', '1', '2', '3', '4', '5']
outputNum = '100' # 출력건수, 기본 10 최대 100
#region = '003002015' # 지역별로 조회하고 싶다면 지역 코드를 지정하면 됨. 여러 지역을 동시에 조회하고 싶다면 쉼표(,)로 코드를 연결해 주면 됨. 예) '003002015,003002014'

contents = ''
for p in page:
    #queryParams = 'pageIndex=' + page + '&display=' + outputNum + '&openApiVlak=' + key + '&srchPolyBizSecd=' + region # 지역별로 조회하게 되면 page를 더 적게 지정해야 할 수도 있음.
    queryParams = 'pageIndex=' + p + '&display=' + outputNum + '&openApiVlak=' + key
    url = 'https://www.youthcenter.go.kr/opi/youthPlcyList.do?'+ queryParams

    # url 불러오기
    response = requests.get(url)
    #print(type(response))
    print(len(response.text)) # 130으로 나오는 경우 해당 페이지에 출력할 정책이 없는 경우임, 즉 130 이전 page까지만 포함시키면 됨.

    #데이터 값 출력해보기
    contents += response.text
    #print(type(contents)) # string 객체로 바뀜

261013
261013
249581
240487
262338
88346


In [37]:
# 데이터 결과값 예쁘게 출력해주는 코드
pp = pprint.PrettyPrinter(indent=4, depth=None)
print(pp.pprint(contents[:3000]))

('<?xml version="1.0" encoding="UTF-8"?>\n'
 '<youthPolicyList>\n'
 '   <pageIndex>1</pageIndex>\n'
 '   <totalCnt>434</totalCnt>\n'
 '   <youthPolicy>\n'
 '      <rnum>1</rnum>\n'
 '      <bizId><![CDATA[R2024022920224]]></bizId>\n'
 '      <polyBizSecd>003002008002</polyBizSecd>\n'
 '      <polyBizTy>지자체</polyBizTy>\n'
 '      <polyBizSjnm><![CDATA[「미취업 청년 지원사업(ALL-Pass)」 대상자 모집 '
 '(성남)]]></polyBizSjnm>\n'
 '      <polyItcnCn><![CDATA[미취업 청년들을 대상으로 구직활동에 필요한 자격증 시험 응시료와 수강료를 '
 '지원]]></polyItcnCn>\n'
 '      <sporCn><![CDATA[○ 지원내용 : 1인 100만원 이내 실비 《 자격증 시험 응시료 및 수강료 》\r\n'
 ' ※ 응시횟수 · 수강횟수 및 신청횟수 제한 없음\r\n'
 '\r\n'
 ' ○ 추가지원 : ‘저소득청년’ / ‘취업애로 청년’은 최대 200만원 지원(1회 100만원 지원 후 (초과시) 100만원 추가 '
 '지원)\r\n'
 '  ※첨부파일상의 추가 지원자격 요건  필수 확인\r\n'
 '\r\n'
 '○ 지원분야 : 어학 20종, 한국사능력검정시험 기본심화, 국가공인자격증 \r\n'
 ' ※세부 지원 자격 분야등에 대해서는 첨부파일 및 신청페이지 참고]]></sporCn>\n'
 '      <sporScvl><![CDATA[null]]></sporScvl>\n'
 '      <bizPrdCn><![CDATA[null]]></bizPrdCn>\n'
 '      <prdRpttSecd><![CDATA[002004]]></p

In [38]:
## 2. BeautifulSoup으로 html parsing
soup = BeautifulSoup(contents, 'html.parser')
#finded_values=soup.find_all('ageInfo')
#[x.text for x in finded_values] # .text를 통해 안의 내용물만을 불러옵니다

/opt/homebrew/lib/python3.11/site-packages/bs4/builder/__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(


In [39]:
print('현재 총 정책 수:', soup.find('totalcnt').text.replace('<totalcnt>', '').replace('</totalcnt>', ''))

현재 총 정책 수: 434


In [40]:
## 3. df 생성
YouthPolicyInfo = {}
# attr 이름 모두 소문자로
#attr_to_find_list=['polybizsjnm','sporcn', 'ageinfo','majrrqiscn','empmsttscn','splzrlmrqiscn','accrrqiscn','prcpcn','aditrscn','prcplmtttrgtcn']
attr_to_find_list=['rnum', 'bizid', 'polybizsecd', 'polybizty', 'polybizsjnm', 'polyitcncn', 'sporcn', 'sporscvl', 'bizprdcn', 'prdrpttsecd', 'rqutprdcn', 
                   'ageinfo','majrrqiscn','empmsttscn','splzrlmrqiscn','accrrqiscn','prcpcn','aditrscn','prcplmtttrgtcn',
                   'rqutproccn', 'pstnpaprcn', 'jdgnprescn', 'rquturla', 'rfcsiteurla1', 'rfcsiteurla2', 'mngtmson', 'mngtmrofchercn', 'cherctpccn',
                   'cnsgnmor', 'tintchercn', 'tintcherctpccn', 'etct', 'polyrlmcd'] # 모두 소문자여야 함.
print(len(attr_to_find_list))

for each_attr in attr_to_find_list:
    finded_attr=soup.find_all(each_attr)
    YouthPolicyInfo[each_attr]=[x.text for x in finded_attr]

df = pd.DataFrame(YouthPolicyInfo)
#alt_colnames = ['정책명', '지원내용', '연령', '전공요건', '취업상태', '특화분화', '학력', '거주지및소득', '추가단서', '참여제한']
alt_colnames = ['번호', '정책 ID', '정책 일련번호', '기관 및 지자체 구분', '정책명', '정책 소개', '지원 내용', '지원 규모', '사업 운영 기간', '사업 신청 기간 반복 구분 코드', '사업 신청 기간', 
                '연령', '전공', '취업 상태', '특화 분화', '학력', '거주지 및 소득', '추가 단서 사항', '참여 제한 대상', 
                '신청 절차', '제출 서류', '심사 발표', '신청 사이트 주소', '참고 사이트 주소1', '참고 사이트 주소2', '주관 부처명', '주관 부처 담당자 이름', '주관 부처 담당자 연락처',
                '운영 기관명', '운영 기관 담당자 이름', '운영 기관 담당자 연락처', '기타 사항', '정책 분야 코드']
print(len(alt_colnames))

df.columns = alt_colnames

33
33


In [41]:
print(df.shape)
df.head()

(534, 33)


,번호,정책 ID,정책 일련번호,기관 및 지자체 구분,정책명,정책 소개,지원 내용,지원 규모,사업 운영 기간,사업 신청 기간 반복 구분 코드,...,참고 사이트 주소1,참고 사이트 주소2,주관 부처명,주관 부처 담당자 이름,주관 부처 담당자 연락처,운영 기관명,운영 기관 담당자 이름,운영 기관 담당자 연락처,기타 사항,정책 분야 코드
0,1,R2024022920224,003002008002,지자체,「미취업 청년 지원사업(ALL-Pass)」 대상자 모집 (성남),미취업 청년들을 대상으로 구직활동에 필요한 자격증 시험 응시료와 수강료를 지원,○ 지원내용 : 1인 100만원 이내 실비 《 자격증 시험 응시료 및 수강료 》\r...,null,null,002004,...,https://www.jobaba.net/sprtPlcy/info/view2020....,https://apply.jobaba.net/bsns/bsnsDetailView.d...,경기도 성남시,null,성남시 콜센터(☎1577-3100) \r\n성남시 청년청소년과(☎031-729-8503),경기도 성남시,null,null,null,023030
1,2,R2024022920243,003002001,지자체,서울시 학자금 대출이자 지원,"경제적 여건이 취약한 서울 거주 대학 재학생, 휴학생 및 졸업생에게 학자금대출 이자...","경제적 여건이 취약한 서울 거주 대학 재학생, 휴학생 및 졸업생에게 학자금대출 이자...","30,000명",-,002004,...,https://youth.seoul.go.kr/youthConts.do?key=23...,-,미래청년기획단,최성원,02-2133-6593,서울특별시,최성원,02-2133-6593,① 본인 외의 자가 신청하는 경우 : 주민등록등본 또는 초본 제출 \r\n② 한국장...,023040
2,3,R2024022920223,003002014,지자체,2024년 청년도전 지원사업,구직단념청년의 구직의욕 고취와 원활한 사회활동 및 노동시장 참여 동기 부여,"□ 지원대상: 구직단념청년* \r\n*6개월 이상 취업 및 교육, 직업 훈련 이력이...","180명(단기 40명, 중기 80명, 장기 60명)",2024-03-01~2024-12-31,002004,...,http://gbyouth.co.kr/policy/list.tc?mn=2379&pa...,-,경상북도,-,경주시청년센터(054-771-9813),경주청년센터,-,경주시청년센터(054-771-9813),-,023030
3,4,R2024022820186,003002009007,지자체,2024 삼척청년센터 청년거버넌스 '청춘파이브' 모집,삼척청년센터는 다양한 창의적 아이디어를 갖춘 청년들이 능동적이고 주체적으로 청년문제...,"*주요내용- 분야별 정기모임 및 워크숍, 자율적인 활동- 전체회의 및 타시도 청년 ...",-청년 100명 내외-모집분야\r\n* 삼청대(삼척청년CEO)\r\n* 삼청기(삼척...,사업기간: 2024.2~12,002004,...,https://xn--660b960a9kaj98a4cy82a.kr/00284/003...,-,삼척청년센터,삼척청년센터,033-572-1839,삼척청년센터,-,-,'청춘파이브' 위촉식 및 오리엔테이션\r\n- 일시: 2024. 3.20.(수) ~...,023050
4,5,R2024022820185,003002015,지자체,(재)경상남도사회서비스원 돌봄이음 서포터즈 2기 모집,경상남도사회서비스원의 돌봄사업을 도민에게 이어주는 돌봄-이음의 역할을 하는 서포터즈,□ 서포터즈 명: 돌봄-이음(약칭 봄이)\r\n- 경상남도사회서비스원의 돌봄 사업을...,제한없음,-,002004,...,-,-,(재)경상남도사회서비스원,봄이 서포터즈 담당자,055-230-8216,주관기관과 동일,-,-,-,023040


In [42]:
## 4. 부족한 정보 추가
# 4-1. 정책 분야 추가
polysec = []
for code in df['정책 분야 코드']:
    if code == '023010':
        polysec.append('일자리 분야')
    elif code == '023020':
        polysec.append('주거 분야')
    elif code == '023030':
        polysec.append('교육 분야')
    elif code == '023040':
        polysec.append('복지·문화 분야')
    elif code == '023050':
        polysec.append('참여·권리 분야')
    else:
        polysec.append('코드 매칭 분야 없음')

df['정책 분야'] = polysec

In [43]:
# 4-2. 온통 청년 url 추가
df['온통 청년 url'] = "https://www.youthcenter.go.kr/youngPlcyUnif/youngPlcyUnifDtl.do?pageIndex=1&frameYn=&bizId=" + df['정책 ID'] + "&dtlOpenYn=&plcyTpOpenTy=&plcyCmprInfo=&srchWord=&chargerOrgCdAll=&srchAge=&trgtJynEmp=&trgtJynEmp=&srchSortOrder=1&pageUnit=12"

In [44]:
print(df.shape)
df.head()

(534, 35)


,번호,정책 ID,정책 일련번호,기관 및 지자체 구분,정책명,정책 소개,지원 내용,지원 규모,사업 운영 기간,사업 신청 기간 반복 구분 코드,...,주관 부처명,주관 부처 담당자 이름,주관 부처 담당자 연락처,운영 기관명,운영 기관 담당자 이름,운영 기관 담당자 연락처,기타 사항,정책 분야 코드,정책 분야,온통 청년 url
0,1,R2024022920224,003002008002,지자체,「미취업 청년 지원사업(ALL-Pass)」 대상자 모집 (성남),미취업 청년들을 대상으로 구직활동에 필요한 자격증 시험 응시료와 수강료를 지원,○ 지원내용 : 1인 100만원 이내 실비 《 자격증 시험 응시료 및 수강료 》\r...,null,null,002004,...,경기도 성남시,null,성남시 콜센터(☎1577-3100) \r\n성남시 청년청소년과(☎031-729-8503),경기도 성남시,null,null,null,023030,교육 분야,https://www.youthcenter.go.kr/youngPlcyUnif/yo...
1,2,R2024022920243,003002001,지자체,서울시 학자금 대출이자 지원,"경제적 여건이 취약한 서울 거주 대학 재학생, 휴학생 및 졸업생에게 학자금대출 이자...","경제적 여건이 취약한 서울 거주 대학 재학생, 휴학생 및 졸업생에게 학자금대출 이자...","30,000명",-,002004,...,미래청년기획단,최성원,02-2133-6593,서울특별시,최성원,02-2133-6593,① 본인 외의 자가 신청하는 경우 : 주민등록등본 또는 초본 제출 \r\n② 한국장...,023040,복지·문화 분야,https://www.youthcenter.go.kr/youngPlcyUnif/yo...
2,3,R2024022920223,003002014,지자체,2024년 청년도전 지원사업,구직단념청년의 구직의욕 고취와 원활한 사회활동 및 노동시장 참여 동기 부여,"□ 지원대상: 구직단념청년* \r\n*6개월 이상 취업 및 교육, 직업 훈련 이력이...","180명(단기 40명, 중기 80명, 장기 60명)",2024-03-01~2024-12-31,002004,...,경상북도,-,경주시청년센터(054-771-9813),경주청년센터,-,경주시청년센터(054-771-9813),-,023030,교육 분야,https://www.youthcenter.go.kr/youngPlcyUnif/yo...
3,4,R2024022820186,003002009007,지자체,2024 삼척청년센터 청년거버넌스 '청춘파이브' 모집,삼척청년센터는 다양한 창의적 아이디어를 갖춘 청년들이 능동적이고 주체적으로 청년문제...,"*주요내용- 분야별 정기모임 및 워크숍, 자율적인 활동- 전체회의 및 타시도 청년 ...",-청년 100명 내외-모집분야\r\n* 삼청대(삼척청년CEO)\r\n* 삼청기(삼척...,사업기간: 2024.2~12,002004,...,삼척청년센터,삼척청년센터,033-572-1839,삼척청년센터,-,-,'청춘파이브' 위촉식 및 오리엔테이션\r\n- 일시: 2024. 3.20.(수) ~...,023050,참여·권리 분야,https://www.youthcenter.go.kr/youngPlcyUnif/yo...
4,5,R2024022820185,003002015,지자체,(재)경상남도사회서비스원 돌봄이음 서포터즈 2기 모집,경상남도사회서비스원의 돌봄사업을 도민에게 이어주는 돌봄-이음의 역할을 하는 서포터즈,□ 서포터즈 명: 돌봄-이음(약칭 봄이)\r\n- 경상남도사회서비스원의 돌봄 사업을...,제한없음,-,002004,...,(재)경상남도사회서비스원,봄이 서포터즈 담당자,055-230-8216,주관기관과 동일,-,-,-,023040,복지·문화 분야,https://www.youthcenter.go.kr/youngPlcyUnif/yo...


In [45]:
# 정책 분야 카테고리 확인
df['정책 분야'].unique()

array(['교육 분야', '복지·문화 분야', '참여·권리 분야', '주거 분야', '일자리 분야'], dtype=object)

In [46]:
## 5. 중복 데이터 제거
# 5-1. 중복 데이터 확인
print(df['정책명'].value_counts())
#df['정책명'].unique()

정책명
2024년 상반기 예술창작공간 입주작가 모집               8
2024년 청년도전 지원사업                        3
「미취업 청년 지원사업(ALL-Pass)」 대상자 모집 (성남)    2
2024년「대학원 대통령과학장학금」신규 장학생 선발 공고        2
청년일자리플랫폼 3D 모델링 & 프린팅 교육 프로그램          2
                                      ..
청년13(일+삶)통장                            1
청년내일저축계좌(청년자산형성지원사업)                   1
청년 자격증 취득비용 지원                         1
대학 ICT연구센터 지원                          1
구직청년 면접 정장 대여                          1
Name: count, Length: 425, dtype: int64


In [47]:
# 5-2. 중복 제거
print(df.shape)
df = df.drop_duplicates()
print(df.shape)

(534, 35)
(434, 35)


In [48]:
## 6. 중앙부처 데이터만 가져오기
# 6-1. 중앙부처/지자체 구분 확인
df['기관 및 지자체 구분'].unique()

array(['지자체', '중앙부처'], dtype=object)

In [49]:
# 6-2. 중앙부처 정책만 있는 df 생성
df_head = df[df['기관 및 지자체 구분']=='중앙부처']

In [50]:
# 7. csv 저장하기
PATH = os.getcwd()
df_head.to_csv(os.path.join(PATH, "중앙부처.csv"))